In [1]:
%load_ext autoreload
%autoreload 2

from Shared.shared import *
from Shared.specific_CNB_sim import *

In [ ]:
sim_folders = [
    f"sim_output/Dopri5_1k_per_pix",
    f"sim_output/Dopri5_2k_per_pix"]

f_names = [
    f"Dopri5_1k_per_pix",
    f"Dopri5_2k_per_pix"]

exclude_nums = [
    20,  # broken halo
    25,  #? halo with only underdensities? look at skymap for this one...
]

## Overdensity band

In [ ]:
for f_name, sim_folder in zip(f_names, sim_folders):

    tot_dens_halos = jnp.array([
        jnp.load(f"{sim_folder}/total_densities_halo{i+1}.npy")
        for i in range(30) if i+1 not in exclude_nums])
    ic(tot_dens_halos.shape)

    SimPlot.overdensity_band(
        dens_arr=tot_dens_halos,
        m_arr=jnp.load(f"{sim_folder}/neutrino_massrange_eV.npy"),
        plot_ylims=(1e-4,1e1),
        fig_path=f"figures/overdensity_band_{f_name}.pdf",
        args=Params()
    )

## Anisotropy Skymaps and Power Spectra

In [ ]:
for f_name, sim_folder in zip(f_names, sim_folders):

    pix_dens_halos = jnp.array([
        jnp.load(f"{sim_folder}/pixel_densities_halo{i+1}.npy")
        for i in range(30) if i+1 not in exclude_nums])
    ic(pix_dens_halos.shape)

    

## Debugging

In [15]:
@chex.dataclass
class SimTest:

    @staticmethod
    @jax.jit
    def number_densities_mass_range(v_arr, m_arr, pix_sr, args):
        
        # Convert velocities to momenta.
        p_arr, _ = Physics.velocities_to_momenta(v_arr, m_arr, args)

        nu_dens = Physics.number_density(
            p_arr[...,0], p_arr[...,-1], pix_sr, args)

        return nu_dens

In [17]:
v_arr = jnp.load(f"sim_output/Dopri5_1k_per_pix/vectors_halo1.npy")[...,3:6]
ic(v_arr.shape)

m_arr = jnp.load(f"sim_output/Dopri5_1k_per_pix/neutrino_massrange_eV.npy")
ic(m_arr.shape)

Npix = 768
nu_per_pix = 1000

# v_arr_re = jnp.reshape(v_arr, (Npix, nu_per_pix, 2, 3))
# nu_dens_pixel = SimTest.jitted_number_densities_all_sky(
#     v_arr_re, m_arr, 4*Params.Pi/Npix, Params())
# ic(nu_dens.shape)

nu_dens_total = SimTest.number_densities_mass_range(
    v_arr, m_arr, 4*Params.Pi, Params()
)
ic(nu_dens_total.shape)

ic| v_arr.shape: (768000, 2, 3)
ic| m_arr.shape: (100,)
ic| nu_dens_total.shape: (100,)


(100,)